# Amazon Bedrock モデルを使用した Strands エージェントを Amazon Bedrock AgentCore Runtime でホスティングする

## 概要

このチュートリアルでは、既存のエージェントを Amazon Bedrock AgentCore Runtime を使用してホスティングする方法を学びます。Amazon Bedrock モデルや Azure OpenAI や Gemini などの非 Bedrock モデルを使用した例を提供します。


### チュートリアルの詳細


| 情報               | 詳細                                                                              |
|:------------------|:---------------------------------------------------------------------------------|
| チュートリアルタイプ   | 会話型                                                                           |
| エージェントタイプ     | シングル                                                                         |
| エージェントフレームワーク | Strands Agents                                                                |
| LLM モデル         | Anthropic Claude Sonnet 4                                                        |
| チュートリアルコンポーネント | AgentCore Runtime でのエージェントホスティング。Strands Agent と Amazon Bedrock モデルの使用 |
| チュートリアル分野     | クロス分野                                                                       |
| 例の複雑さ          | 簡単                                                                             |
| 使用する SDK        | Amazon BedrockAgentCore Python SDK と boto3                                      |

### チュートリアルのアーキテクチャ

このチュートリアルでは、既存のエージェントを AgentCore ランタイムにデプロイする方法について説明します。

デモンストレーションのために、Amazon Bedrock モデルを使用した Strands Agent を使用します。

この例では、`get_weather` と `get_time` という 2 つのツールを持つ非常にシンプルなエージェントを使用します。

<div style="text-align:left">
    <img src="images/architecture_runtime.png" width="50%"/>
</div>

### チュートリアルの主な機能

* Amazon Bedrock AgentCore Runtime でのエージェントのホスティング
* Amazon Bedrock モデルの使用
* Strands Agents の使用

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Python 3.10+
* AWS 認証情報
* Amazon Bedrock AgentCore SDK
* Strands Agents
* Docker の実行環境

In [ ]:
#!uv add -r requirements.txt --active

import os
 #os.environ['AWS_PROFILE'] = 'your_profile_name'
os.environ['AWS_PROFILE'] = 'cline2'

## エージェントの作成とローカルでの実験

AgentCore Runtime にエージェントをデプロイする前に、まずローカルで開発して実験目的で実行してみましょう。

本番環境のエージェントアプリケーションでは、エージェント作成プロセスとエージェント呼び出しプロセスを分離する必要があります。AgentCore Runtime では、エージェントの呼び出し部分を `@app.entrypoint` デコレータで装飾し、ランタイムのエントリポイントとします。まず、実験段階で各エージェントがどのように開発されるかを見てみましょう。

ここでのアーキテクチャは次のようになります：

<div style="text-align:left">
    <img src="images/architecture_local.png" width="50%"/>
</div>

In [7]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # calculator ツールをインポート
import argparse
import json
from strands.models import BedrockModel

# カスタムツールの作成
@tool
def weather():
    """ 天気を取得 """ # ダミー実装
    return "sunny"


model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"
# model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_bedrock(payload):
    """
    ペイロードでエージェントを呼び出す
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_bedrock(json.loads(args.payload))
    print(response)

Overwriting strands_claude.py


#### ローカルエージェントの呼び出し

In [8]:
!python strands_claude.py '{"prompt": "今の天気は?"}'

現在の天気情報を取得いたします。
Tool #1: weather
現在の天気は「晴れ(sunny)」です。お出かけには良い天気ですね。現在の天気は「晴れ(sunny)」です。お出かけには良い天気ですね。


## AgentCore Runtime へのデプロイ準備

それでは、エージェントを AgentCore Runtime にデプロイしましょう。そのためには以下が必要です：
* `from bedrock_agentcore.runtime import BedrockAgentCoreApp` で Runtime App をインポート
* `app = BedrockAgentCoreApp()` でコード内に App を初期化
* `@app.entrypoint` デコレータで呼び出し関数を装飾
* `app.run()` で AgentCoreRuntime にエージェントの実行を制御させる

### Amazon Bedrock モデルを使用した Strands Agents
まずは Amazon Bedrock モデルを使用した Strands Agent から始めましょう。他のすべても全く同じように動作します。

In [28]:
%%writefile strands_claude.py
from strands import Agent, tool
from strands_tools import calculator # calculator ツールをインポート
import argparse
import json
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from strands.models import BedrockModel
import os

app = BedrockAgentCoreApp()

# カスタムツールの作成
@tool
def weather():
    """ 天気を取得 """ # ダミー実装
    return "sunny"


model_id = "us.anthropic.claude-sonnet-4-20250514-v1:0"
model = BedrockModel(
    model_id=model_id,
)
agent = Agent(
    model=model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

@app.entrypoint
def strands_agent_bedrock(payload):
    """
    ペイロードでエージェントを呼び出す
    """
    user_input = payload.get("prompt")
    print("User input:", user_input)
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    app.run()

Overwriting strands_claude.py


## 裏側で何が起こっているか

`BedrockAgentCoreApp` を使用すると、自動的に以下のことが行われます：

* ポート 8080 でリッスンする HTTP サーバーを作成
* エージェントの要件を処理するための `/invocations` エンドポイントを実装
* ヘルスチェック用の `/ping` エンドポイントを実装（非同期エージェントには非常に重要）
* 適切なコンテンツタイプとレスポンス形式を処理
* AWS 標準に従ったエラー処理を管理

## エージェントを AgentCore Runtime にデプロイする

`CreateAgentRuntime` 操作は包括的な設定オプションをサポートしており、コンテナイメージ、環境変数、暗号化設定を指定できます。また、クライアントがエージェントと通信する方法を制御するためのプロトコル設定（HTTP、MCP）や認証メカニズムも設定できます。

**注意:** 運用のベストプラクティスは、CI/CD パイプラインと IaC を使用してコードをコンテナとしてパッケージ化し、ECR にプッシュすることです。

このチュートリアルでは、Amazon Bedrock AgentCode Python SDK を使用して、アーティファクトを簡単にパッケージ化し、AgentCore ランタイムにデプロイします。

### ランタイムロールの作成

まず、AgentCore Runtime 用の IAM ロールを作成しましょう。これは、あらかじめ開発されたユーティリティ関数を使用して行います。

In [29]:
import sys
import os

# 現在のノートブックのディレクトリを取得
current_dir = os.path.dirname(os.path.abspath('__file__' if '__file__' in globals() else '.'))

# utils.py の場所に移動
utils_dir = os.path.join(current_dir, '..')
utils_dir = os.path.abspath(utils_dir)

# sys.path に追加
sys.path.insert(0, utils_dir)

from utils import create_agentcore_role

agent_name="strands_claude"
agentcore_iam_role = create_agentcore_role(agent_name=agent_name)

attaching role policy agentcore-strands_claude-role


### AgentCore Runtime デプロイの設定

次に、スターターツールキットを使用して、エントリポイント、先ほど作成した実行ロール、および要件ファイルを使用して AgentCore Runtime デプロイを設定します。また、起動時に Amazon ECR リポジトリを自動作成するようにスターターキットを設定します。

設定ステップでは、アプリケーションコードに基づいて Docker ファイルが生成されます。

<div style="text-align:left">
    <img src="images/configure.png" width="40%"/>
</div>

In [35]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()

response = agentcore_runtime.configure(
    entrypoint="strands_claude.py",
    execution_role=agentcore_iam_role['Role']['Arn'],
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name=agent_name+"2"
)
response

⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Bedrock AgentCore configured: /home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.bedrock_agentcore.yaml


ConfigureResult(config_path=PosixPath('/home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.bedrock_agentcore.yaml'), dockerfile_path=PosixPath('/home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/Dockerfile'), dockerignore_path=PosixPath('/home/coder/amazon-bedrock-agentcore-samples/01-tutorials/01-AgentCore-runtime/01-hosting-agent/01-strands-with-bedrock-model/.dockerignore'), runtime='Docker', region='us-east-1', account_id='320462930492', execution_role='arn:aws:iam::320462930492:role/agentcore-strands_claude-role', ecr_repository=None, auto_create_ecr=True)

### AgentCore Runtime へのエージェントの起動

Docker ファイルができたので、エージェントを AgentCore Runtime に起動しましょう。これにより、Amazon ECR リポジトリと AgentCore Runtime が作成されます。

<div style="text-align:left">
    <img src="images/launch.png" width="75%"/>
</div>

In [40]:
launch_result = agentcore_runtime.launch()
launch_result

Build: #10 9.802 ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.


The push refers to repository [320462930492.dkr.ecr.us-east-1.amazonaws.com/bedrock_agentcore-strands_claude2]
1926f4b05988: Preparing
a83b51ec752e: Preparing
e930bf0031c8: Preparing
42730df720ac: Preparing
aa3c12b4e2d7: Preparing
7e96e04df626: Preparing
f6dc80d9d167: Preparing
3ba4af0751c6: Preparing
d379451f91b6: Preparing
7cc7fe68eff6: Preparing
7e96e04df626: Waiting
f6dc80d9d167: Waiting
3ba4af0751c6: Waiting
d379451f91b6: Waiting
7cc7fe68eff6: Waiting
e930bf0031c8: Pushed
1926f4b05988: Pushed
aa3c12b4e2d7: Pushed
7e96e04df626: Pushed
f6dc80d9d167: Pushed
a83b51ec752e: Pushed
d379451f91b6: Pushed
3ba4af0751c6: Pushed
7cc7fe68eff6: Pushed
42730df720ac: Pushed
latest: digest: sha256:34febf5f1cbb7d63ea1b5dd9ef5dd4e7a0cb8e77cad5adcf2f57f261f16d6684 size: 2414


Deployed to cloud: arn:aws:bedrock-agentcore:us-east-1:320462930492:runtime/strands_claude2-dMXCq2GOUP
🔍 Agent logs available at:
   /aws/bedrock-agentcore/runtimes/strands_claude2-dMXCq2GOUP-DEFAULT
   /aws/bedrock-agentcore/runtimes/strands_claude2-dMXCq2GOUP-DEFAULT/runtime-logs
💡 Tail logs with: aws logs tail /aws/bedrock-agentcore/runtimes/strands_claude2-dMXCq2GOUP-DEFAULT --follow
💡 Or view recent logs: aws logs tail /aws/bedrock-agentcore/runtimes/strands_claude2-dMXCq2GOUP-DEFAULT --since 1h


LaunchResult(mode='cloud', tag='bedrock_agentcore-strands_claude2:latest', env_vars=None, port=None, runtime=None, ecr_uri='320462930492.dkr.ecr.us-east-1.amazonaws.com/bedrock_agentcore-strands_claude2', agent_id='strands_claude2-dMXCq2GOUP', agent_arn='arn:aws:bedrock-agentcore:us-east-1:320462930492:runtime/strands_claude2-dMXCq2GOUP', build_output=['#0 building with "default" instance using docker driver', '#1 [internal] load build definition from Dockerfile', '#1 transferring dockerfile: 895B done', '#1 WARN: FromPlatformFlagConstDisallowed: FROM --platform flag should not use constant value "linux/amd64" (line 1)', '#1 DONE 0.0s', '#2 [internal] load metadata for public.ecr.aws/docker/library/python:3.10-slim', '#2 DONE 0.2s', '#3 [internal] load .dockerignore', '#3 transferring context: 744B done', '#3 DONE 0.0s', '#4 [1/7] FROM public.ecr.aws/docker/library/python:3.10-slim@sha256:81f1cdb3770d54ecfdbddcc52c2125fce674c14a1d976dfd8f65dc0734f9c3c5', '#4 DONE 0.0s', '#5 [internal] 

### AgentCore Runtime のステータス確認
AgentCore Runtime をデプロイしたので、そのデプロイステータスを確認しましょう。

In [ ]:
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

### Invoking AgentCore Runtime

Finally, we can invoke our AgentCore Runtime with a payload

<div style="text-align:left">
    <img src="images/invoke.png" width=75%"/>
</div>

In [ ]:
invoke_response = agentcore_runtime.invoke({"prompt": "How is the weather now?"})
invoke_response

### Processing invocation results

We can now process our invocation results to include it in an application

In [ ]:
from IPython.display import Markdown, display
import json
response_text = json.loads(invoke_response['response'][0].decode("utf-8"))
display(Markdown(response_text))

### Invoking AgentCore Runtime with boto3

Now that your AgentCore Runtime was created you can invoke it with any AWS SDK. For instance, you can use the boto3 `invoke_agent_runtime` method for it.

In [ ]:
import boto3
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "What is 2+2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                logger.info(line)
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))

## Cleanup (Optional)

Let's now clean up the AgentCore Runtime created

In [ ]:
launch_result

In [ ]:
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)
ecr_client = boto3.client(
    'ecr',
    region_name=region
    
)

iam_client = boto3.client('iam')

runtime_delete_response = agentcore_control_client.delete_agent_runtime(
    agentRuntimeId=launch_result.agent_id,
    
)

response = ecr_client.delete_repository(
    repositoryName=launch_result.ecr_uri.split('/')[1],
    force=True
)

policies = iam_client.list_role_policies(
    RoleName=agentcore_iam_role['Role']['RoleName'],
    MaxItems=100
)

for policy_name in policies['PolicyNames']:
    iam_client.delete_role_policy(
        RoleName=agentcore_iam_role['Role']['RoleName'],
        PolicyName=policy_name
    )
iam_response = iam_client.delete_role(
    RoleName=agentcore_iam_role['Role']['RoleName']
)

# Congratulations!